# Ag-Analytics® Land Value AI - API Demo 

The Ag-Analytics® Land Value AI provides timely and accurately estimation of any given parcel on any specific day. Current version of Land Value has two types of estimations: Auto Comps takes the comparables referring to properties with characteristics that are similar to the target parcel whose value is being estimated. AI models were trained on the real historical and near-real time climate, agriculture, and economic data. It provides a science-based and big-data-based estimations on the parcel value.

### Required libraries

In [11]:
import requests
import json
import pandas as pd

### Request Parameter Details

1). __aoi__ (GeoJSON in String, required): Geometry of area of Interest in GeoJSON format. Need to be passed into the API as string

       
2). __model__(string): The Model used for Land Value estimation.Models available in current version are:
    
    i. Linear: Multiple Linear Regression
    
    ii. Tree: Decision Tree Regression 
    
    iii. RF: Random Forest Regression
    
    iv. Auto: Auto Comps


3). __Sale_Date(string)__: Expected sale date in ‘yyyy-mm-dd’ format. EX: ‘2020-04-20’


### Request in Json

The Land Value AI API takes the request in Json format. Below are examples formatted in Json. 

In [8]:
#Request Example
aoi = "{\"type\":\"Feature\",\"properties\":{},\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-95.84165811538698,46.30405964827517],[-95.84174394607544,46.29697349504195],[-95.83659410476686,46.29691419281915],[-95.83655118942261,46.29814470078284],[-95.83831071853638,46.29817435123596],[-95.83837509155275,46.3029330408763],[-95.83831071853638,46.30303680831659],[-95.83801031112672,46.30296268873647],[-95.837699174881,46.30297010069899],[-95.83699107170106,46.30349634747375],[-95.83695888519289,46.30367423256352],[-95.8370339870453,46.30393364728336],[-95.83717346191408,46.3040003537269],[-95.8375597000122,46.30405223646016],[-95.8375597000122,46.30318504717584],[-95.84077835083009,46.30317022330798],[-95.84081053733827,46.30408188371422],[-95.84165811538698,46.30405964827517]]]}}"
model = "RF"
Sale_Date = "2020-04-15"
values = {"aoi":aoi,
          "model":model,
          "Sale_Date":Sale_Date
    
}
# Basic Header Pattern.
headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

### API Function

In [18]:
def LandValueAI(values,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/land-value-ai'
        
        response = requests.post(url, json = values, headers = headers).json()
        
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

#### 1. Example with AI Models

In [10]:
# Calling API with Random Forest Model
values["model"] = "RF"
LandValueResponse = LandValueAI(values,headers)

{'Result': {'Model': 'RF', 'Parcel_Area': 51.74941101442054, 'Price_Acre': 10520.09, 'Sale_Date': '2020-04-15', 'Total_Price': 544408.4613186953, 'Unit': '$'}, 'status': 'SUCCESS'}


##### Display result if the API succeed.

In [17]:
result_df = pd.DataFrame()
if LandValueResponse['status'] == 'SUCCESS':
    result = LandValueResponse['Result']
    result_df = pd.DataFrame([result])
result_df.T

,0
Model,RF
Parcel_Area,51.7494
Price_Acre,10520.1
Sale_Date,2020-04-15
Total_Price,544408
Unit,$


#### 2. Example with Auto Comps

In [19]:
# Calling API with Auto Comps Model
values["model"] = "Auto"
LandValueResponse = LandValueAI(values,headers)

{'Model': 'Auto', 'Result': {'For Sale': {'Percentile_10': 1285.0, 'Percentile_90': 13255.0, 'avg_price': 4792.1977823893085, 'max_price': 895000.0, 'min_price': 26.0, 'num_list': 811}, 'Parcel_Area': 51.74941101442054, 'Sale_Date': '2020-04-15', 'Sold': {'Percentile_10': 3549.0, 'Percentile_90': 11100.0, 'avg_price': 6879.141775894334, 'max_price': 69371.0, 'min_price': 792.0, 'num_list': 318}, 'Unit': '$'}, 'status': 'SUCCESS'}


##### Display result if the API succeed.

In [28]:
result_df = pd.DataFrame()
if LandValueResponse['status'] == 'SUCCESS':
    result_sale = pd.DataFrame([LandValueResponse['Result']['For Sale']],index=['For Sale'])
    result_sold= pd.DataFrame([LandValueResponse['Result']['Sold']],index=['Sold'])
    result_df = pd.concat([result_sale,result_sold])
result_df.T

,For Sale,Sold
Percentile_10,1285.000000,3549.000000
Percentile_90,13255.000000,11100.000000
avg_price,4792.197782,6879.141776
max_price,895000.000000,69371.000000
min_price,26.000000,792.000000
num_list,811.000000,318.000000
